In [2]:
from __future__ import division, absolute_import

import os
import sys
import random

import numpy as np
import datetime

#root
absPath = '/home/angela/padding_EBI/'
sys.path.insert(0, absPath)

#from src.Target import Target

np.random.seed(8)
random.seed(8)

from src.preprocessing import *
from src.model_architecture import *
from src.training_model import *
from src.postprocessing import *
from src.comparing_results import *
#from src.callbacks import *
%matplotlib inline

### Defining parameters

In [3]:
#list_paddings = ['post_padding', 'pre_padding', 'mid_padding', 'stretch_padding', 'ext_padding', 
#                 'rdm_padding', 'aug_padding']
list_paddings = ['post_padding', 'pre_padding', 'mid_padding', 'strf_padding', 'ext_padding', 
                 'rnd_padding']
#hierarchy of folders: annotation/dataset/architecture/n_neurs/task/padding
folder = 'EC_number/archaea/3denses/bio_neurons/'
column = "EC number"

n_folds = 10

dicti = creating_dict()

max_lenn = 1000

n_class = 2 #number of classes to output
drop_per = 0.2 #Input dropout 
n_neur = [314,77, 8]
drop_hid = 0.5
dict_size = len(dicti)

batch_size = 54
epochss = 200

callbacks_list = [None, None, None, None, False, False, False, False]
task = "task1/"
architecture = "only_denses"

### Loading stuff

In [4]:
file_idcs_aug = os.path.join(absPath, 'data/', folder, 'idcs_aug_split.pickle')

with open(file_idcs_aug, "rb") as input_file:
    k_aug_indices = pickle.load(input_file)
    
file_idcs = os.path.join(absPath, 'data/', folder, 'idcs_split.pickle')

with open(file_idcs, "rb") as input_file:
    splitting_sets = pickle.load(input_file)

# Defining Deep Learning model  - Task 1

In [5]:
model = building_3dense_model_task1(max_lenn, dict_size, n_neur, n_class, drop_per, drop_hid, 'softmax', folder)

generators_dict = {} 
for model_type in list_paddings:
    generators_dict[model_type] = trainval_generators(splitting_sets, k_aug_indices, model_type, 
                                                      folder, batch_size, 'labels_task1', True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000, 26)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000, 26)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 26000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 314)               8164314   
_________________________________________________________________
dropout_2 (Dropout)          (None, 314)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 77)                24255     
_________________________________________________________________
dropout_3 (Dropout)          (None, 77)                0         
__________

### Training model

In [ ]:
for model_type in list_paddings:
    generators = generators_dict[model_type]
    for idx,i in enumerate(generators):
        for idx,i in enumerate(generators):
            if model_type == "aug_padding":
                i_train, i_val, i_test = k_aug_indices[idx]
            else: 
                i_train, i_val, i_test = splitting_sets[idx]
            len_train, len_val, len_test = len(i_train), len(i_val), len(i_test)
            train_generator, val_generator = i
            model_training(model_type, folder, task, idx, train_generator, val_generator,
                           callbacks_list, architecture, max_len, dict_size,
                           n_neur, n_class, drop_per, drop_hid, final_act, epochss, 
                           n_filt = None, kernel_size=None, pool_size=None)

In [ ]:
# BORRAR CUANDO COMPRUEBE QUE LO OTRO VA BIEN


task = "task1/"
for model_type in list_paddings:
    generators = generators_dict[model_type]
    for idx,i in enumerate(generators):
        if model_type == "aug_padding":
            i_train, i_val, i_test = k_aug_indices[idx]
        else:
            i_train, i_val, i_test = splitting_sets[idx]
        len_train, len_val, len_test = len(i_train), len(i_val), len(i_test)
        train_generator, val_generator = i
        folder_cp = ''.join(string for string in [folder, task, model_type, '/', str(idx)]) 
        #print(folder_cp)
        if not os.path.exists(os.path.join(absPath, 'data/checkpoint/', folder_cp)):
            os.makedirs(''.join(string for string in [absPath, 'data/checkpoint/', folder_cp]))

        callbacks_list = calling_callbacks(folder_cp, folder, model_type, None, None, None, 
                                   None, False, False, False, False)
        folder_task1 =  ''.join(string for string in [folder, task])
        model = building_2dense_model_task2(max_lenn, dict_size, n_neur, n_class, drop_per, drop_hid, 'softmax', folder)
        #writing log file 
        log_file = ''.join(string for string in [absPath, 'data/checkpoint/', folder, task, 'log_file.txt' ]) 
        f = open(log_file, 'a+')
        print('Model type: %s \n' % model_type, file=f)
        print('Fold: %i \n' % idx, file=f)
 
        start = time.time()
        formatted_time = datetime.datetime.now()
        print('Starting time: %s \n' % formatted_time, file=f)
        history = model.fit_generator(generator=train_generator, 
                              validation_data=val_generator,
                             steps_per_epoch= int(len_train/batch_size),
                              validation_steps=int(len_val/batch_size),
                             epochs=epochss,
                             callbacks=callbacks_list,
                             verbose=1)
        end = time.time()
        formatted_endtime = datetime.datetime.now()
        print('Finishing time: %s \n' % formatted_endtime, file=f)
        count_time(start, end, folder, model_type)
        saving_results(history, model_type, folder, task, idx, True)
        f.close()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1000, 26)          0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 1000, 26)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 26000)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 314)               8164314   
_________________________________________________________________
dropout_6 (Dropout)          (None, 314)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 77)                24255     
_________________________________________________________________
dropout_7 (Dropout)          (None, 77)                0         
__________


Epoch 00069: val_acc did not improve from 0.88015
Epoch 70/200
251/251 [==============================] - 4s 15ms/step - loss: 0.3199 - acc: 0.8609 - val_loss: 0.4007 - val_acc: 0.8529

Epoch 00070: val_acc did not improve from 0.88015
Epoch 71/200
251/251 [==============================] - 4s 15ms/step - loss: 0.3357 - acc: 0.8535 - val_loss: 0.4034 - val_acc: 0.8539

Epoch 00071: val_acc did not improve from 0.88015
Epoch 72/200
251/251 [==============================] - 4s 15ms/step - loss: 0.3213 - acc: 0.8631 - val_loss: 0.4019 - val_acc: 0.8508

Epoch 00072: val_acc did not improve from 0.88015
Epoch 73/200
251/251 [==============================] - 4s 15ms/step - loss: 0.3233 - acc: 0.8604 - val_loss: 0.3839 - val_acc: 0.8655

Epoch 00073: val_acc did not improve from 0.88015
Epoch 74/200
251/251 [==============================] - 4s 15ms/step - loss: 0.3241 - acc: 0.8589 - val_loss: 0.3960 - val_acc: 0.8543

Epoch 00074: val_acc did not improve from 0.88015
Epoch 75/200
251/25

251/251 [==============================] - 4s 15ms/step - loss: 0.2559 - acc: 0.8977 - val_loss: 0.2717 - val_acc: 0.9228

Epoch 00152: val_acc did not improve from 0.92802
Epoch 153/200
251/251 [==============================] - 4s 15ms/step - loss: 0.2626 - acc: 0.8936 - val_loss: 0.2918 - val_acc: 0.9095

Epoch 00153: val_acc did not improve from 0.92802
Epoch 154/200
251/251 [==============================] - 4s 15ms/step - loss: 0.2550 - acc: 0.8992 - val_loss: 0.2972 - val_acc: 0.9018

Epoch 00154: val_acc did not improve from 0.92802
Epoch 155/200
251/251 [==============================] - 4s 15ms/step - loss: 0.2588 - acc: 0.8958 - val_loss: 0.3006 - val_acc: 0.9015

Epoch 00155: val_acc did not improve from 0.92802
Epoch 156/200
251/251 [==============================] - 4s 15ms/step - loss: 0.2584 - acc: 0.8971 - val_loss: 0.2731 - val_acc: 0.9238

Epoch 00156: val_acc did not improve from 0.92802
Epoch 157/200
251/251 [==============================] - 4s 15ms/step - loss: 0

251/251 [==============================] - 4s 15ms/step - loss: 0.2347 - acc: 0.9064 - val_loss: 0.2513 - val_acc: 0.9266

Epoch 00195: val_acc did not improve from 0.92977
Epoch 196/200
251/251 [==============================] - 4s 15ms/step - loss: 0.2244 - acc: 0.9110 - val_loss: 0.2346 - val_acc: 0.9266

Epoch 00196: val_acc did not improve from 0.92977
Epoch 197/200
251/251 [==============================] - 4s 15ms/step - loss: 0.2304 - acc: 0.9072 - val_loss: 0.2222 - val_acc: 0.9277

Epoch 00197: val_acc did not improve from 0.92977
Epoch 198/200
251/251 [==============================] - 4s 15ms/step - loss: 0.2301 - acc: 0.9090 - val_loss: 0.2382 - val_acc: 0.9179

Epoch 00198: val_acc did not improve from 0.92977
Epoch 199/200
251/251 [==============================] - 4s 15ms/step - loss: 0.2394 - acc: 0.9045 - val_loss: 0.2564 - val_acc: 0.9189

Epoch 00199: val_acc did not improve from 0.92977
Epoch 200/200
251/251 [==============================] - 4s 15ms/step - loss: 0

251/251 [==============================] - 4s 15ms/step - loss: 0.5131 - acc: 0.7247 - val_loss: 0.5912 - val_acc: 0.7348

Epoch 00030: val_acc did not improve from 0.80259
Epoch 31/200
251/251 [==============================] - 4s 15ms/step - loss: 0.5042 - acc: 0.7423 - val_loss: 0.5874 - val_acc: 0.7425

Epoch 00031: val_acc did not improve from 0.80259
Epoch 32/200
251/251 [==============================] - 4s 15ms/step - loss: 0.4755 - acc: 0.7647 - val_loss: 0.5748 - val_acc: 0.7505

Epoch 00032: val_acc did not improve from 0.80259
Epoch 33/200
251/251 [==============================] - 4s 15ms/step - loss: 0.5097 - acc: 0.7278 - val_loss: 0.5847 - val_acc: 0.7376

Epoch 00033: val_acc did not improve from 0.80259
Epoch 34/200
251/251 [==============================] - 4s 15ms/step - loss: 0.4740 - acc: 0.7573 - val_loss: 0.5750 - val_acc: 0.7411

Epoch 00034: val_acc did not improve from 0.80259
Epoch 35/200
251/251 [==============================] - 4s 15ms/step - loss: 0.4995

...

251/251 [==============================] - 4s 15ms/step - loss: 0.5129 - acc: 0.7306 - val_loss: 0.6027 - val_acc: 0.7327

Epoch 00028: val_acc did not improve from 0.77498
Epoch 29/200
251/251 [==============================] - 4s 15ms/step - loss: 0.5252 - acc: 0.7116 - val_loss: 0.6244 - val_acc: 0.7271

Epoch 00029: val_acc did not improve from 0.77498
Epoch 30/200
251/251 [==============================] - 4s 15ms/step - loss: 0.5166 - acc: 0.7385 - val_loss: 0.5892 - val_acc: 0.7362

Epoch 00030: val_acc did not improve from 0.77498
Epoch 31/200
251/251 [==============================] - 4s 15ms/step - loss: 0.4875 - acc: 0.7534 - val_loss: 0.5818 - val_acc: 0.7404

Epoch 00031: val_acc did not improve from 0.77498
Epoch 32/200
251/251 [==============================] - 4s 15ms/step - loss: 0.4877 - acc: 0.7598 - val_loss: 0.5729 - val_acc: 0.7474

Epoch 00032: val_acc did not improve from 0.77498
Epoch 33/200
251/251 [==============================] - 4s 15ms/step - loss: 0.5211


Epoch 00063: val_acc did not improve from 0.83578
Epoch 64/200
251/251 [==============================] - 4s 15ms/step - loss: 0.3660 - acc: 0.8295 - val_loss: 0.4341 - val_acc: 0.8421

Epoch 00064: val_acc improved from 0.83578 to 0.84207, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons/task1/post_padding/5/weights-improvement-064-0.8421.hdf5
Epoch 65/200
251/251 [==============================] - 4s 15ms/step - loss: 0.3733 - acc: 0.8245 - val_loss: 0.4415 - val_acc: 0.8382

Epoch 00065: val_acc did not improve from 0.84207
Epoch 66/200
251/251 [==============================] - 4s 15ms/step - loss: 0.3541 - acc: 0.8394 - val_loss: 0.4380 - val_acc: 0.8365

Epoch 00066: val_acc did not improve from 0.84207
Epoch 67/200
251/251 [==============================] - 4s 15ms/step - loss: 0.3483 - acc: 0.8413 - val_loss: 0.4374 - val_acc: 0.8368

Epoch 00067: val_acc did not improve from 0.84207
Epoch 68/200
251/251 [==========================

251/251 [==============================] - 4s 15ms/step - loss: 0.2916 - acc: 0.8792 - val_loss: 0.3689 - val_acc: 0.8690

Epoch 00101: val_acc did not improve from 0.88994
Epoch 102/200
251/251 [==============================] - 4s 15ms/step - loss: 0.2944 - acc: 0.8758 - val_loss: 0.3428 - val_acc: 0.8875

Epoch 00102: val_acc did not improve from 0.88994
Epoch 103/200
251/251 [==============================] - 4s 15ms/step - loss: 0.2992 - acc: 0.8724 - val_loss: 0.3404 - val_acc: 0.8903

Epoch 00103: val_acc improved from 0.88994 to 0.89029, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons/task1/post_padding/5/weights-improvement-103-0.8903.hdf5
Epoch 104/200
251/251 [==============================] - 4s 15ms/step - loss: 0.2898 - acc: 0.8783 - val_loss: 0.3394 - val_acc: 0.8903

Epoch 00104: val_acc did not improve from 0.89029
Epoch 105/200
251/251 [==============================] - 4s 15ms/step - loss: 0.2895 - acc: 0.8761 - val_loss

251/251 [==============================] - 4s 15ms/step - loss: 0.2615 - acc: 0.8960 - val_loss: 0.2930 - val_acc: 0.9126

Epoch 00140: val_acc did not improve from 0.91894
Epoch 141/200
251/251 [==============================] - 4s 15ms/step - loss: 0.2596 - acc: 0.8952 - val_loss: 0.2792 - val_acc: 0.9214

Epoch 00141: val_acc improved from 0.91894 to 0.92138, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons/task1/post_padding/5/weights-improvement-141-0.9214.hdf5
Epoch 142/200
251/251 [==============================] - 4s 15ms/step - loss: 0.2596 - acc: 0.8949 - val_loss: 0.2911 - val_acc: 0.9099

Epoch 00142: val_acc did not improve from 0.92138
Epoch 143/200
251/251 [==============================] - 4s 15ms/step - loss: 0.2542 - acc: 0.9000 - val_loss: 0.2813 - val_acc: 0.9203

Epoch 00143: val_acc did not improve from 0.92138
Epoch 144/200
251/251 [==============================] - 4s 15ms/step - loss: 0.2631 - acc: 0.8951 - val_loss


Epoch 00179: val_acc improved from 0.92732 to 0.92942, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons/task1/post_padding/5/weights-improvement-179-0.9294.hdf5
Epoch 180/200
251/251 [==============================] - 4s 15ms/step - loss: 0.2323 - acc: 0.9109 - val_loss: 0.2608 - val_acc: 0.9249

Epoch 00180: val_acc did not improve from 0.92942
Epoch 181/200
251/251 [==============================] - 4s 15ms/step - loss: 0.2365 - acc: 0.9080 - val_loss: 0.2580 - val_acc: 0.9273

Epoch 00181: val_acc did not improve from 0.92942
Epoch 182/200
251/251 [==============================] - 4s 15ms/step - loss: 0.2322 - acc: 0.9111 - val_loss: 0.2565 - val_acc: 0.9242

Epoch 00182: val_acc did not improve from 0.92942
Epoch 183/200
251/251 [==============================] - 4s 15ms/step - loss: 0.2388 - acc: 0.9078 - val_loss: 0.2601 - val_acc: 0.9242

Epoch 00183: val_acc did not improve from 0.92942
Epoch 184/200
251/251 [=====================